<a href="https://colab.research.google.com/github/Vamarinpe/PAE/blob/master/CodRecurso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Por agente

# importa la librería
import pandas as pd
import numpy as np
pd.set_option('display.notebook_repr_html', False)
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import csv

Datos = pd.read_csv(
    "https://raw.githubusercontent.com/Vamarinpe/PAE/master/Recurso.csv",
    sep = ',',         # separador de campos
    thousands = None,  # separador de miles para números
    decimal = '.',
    index_col=["Fecha","Codigo Agente","Recurso"],
    usecols=["Fecha","Codigo Agente","Recurso","Precio Oferta Ideal"],      # Datosnuevo=Datos[['Fecha','Codigo Agente','Recurso']]  - otra forma
    parse_dates=["Fecha"],
    dayfirst=True)


#Datos = pd.read_csv(
    #"https://raw.githubusercontent.com/Vamarinpe/Ensayo2/master/Datos.csv",
    #sep = ',',         # separador de campos
    #thousands = None,  # separador de miles para números
    #decimal = '.',
    #quoting=csv.QUOTE_NONNUMERIC)

#Datos['Fecha'] = Datos.Fecha\
                             #.map(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [0]:
len(Datos)
Datos.columns
Datos.describe()
Datos.head(60)

In [0]:
Datos.count()

In [0]:
Datos.groupby('Codigo Agente')['Precio Oferta Ideal'].count()
Datos.groupby('Recurso')['Precio Oferta Ideal'].count()
Datos.groupby('Fecha')['Precio Oferta Ideal'].count()            # también sin el ['Precio Oferta Ideal']

In [0]:
Datos.loc[('2012-09-04')]
Datos.loc[('2012-09-04','EPMG')]
plt.plot(Datos.loc[('2017-09-05','EPMG')])
plt.plot(Datos.loc[('2017-09-06','EPMG')])
plt.plot(Datos.loc[('2019-08-31','EPMG')])
plt.show()

In [0]:
Datos.loc['2012-01-01':'2013-12-31']

In [0]:
Datos.xs('2012-06-08',level='Fecha')
Datos.xs('EPMG',level='Codigo Agente')
Datos.xs('GUATAPE',level='Recurso').plot()
Datos.xs('PLAYAS',level='Recurso').plot()
plt.rcParams["figure.figsize"] = (20,8)


In [0]:
Datos.iloc[5]

In [0]:
def capacidad(x):
  Cap = pd.read_csv(
    x,
    sep = ';',         # separador de campos
    thousands = None,  # separador de miles para números
    decimal = '.',
    index_col=["Recurso"],
    usecols=["Recurso","Capacidad Efectiva Neta kW"])
  return Cap.sort_values(by='Capacidad Efectiva Neta kW',ascending=False).drop_duplicates().head(10)

C2000=capacidad("/content/Capacidad_Efectiva_Neta_(kW)_2000.csv")
C2001=capacidad("/content/Capacidad_Efectiva_Neta_(kW)_2001.csv")


In [17]:
Datos.xs(C2001.index[4],level='Recurso')
  

                          Precio Oferta Ideal
Fecha      Codigo Agente                     
2000-01-01 CHVG                         48.37
2000-01-02 CHVG                         50.95
2000-01-03 CHVG                         50.16
2000-01-04 CHVG                         50.16
2000-01-05 CHVG                         47.28
2000-01-06 CHVG                         37.78
2000-01-07 CHVG                         29.13
2000-01-08 CHVG                         42.28
2000-01-09 CHVG                         41.62
2000-01-10 CHVG                         39.20
2000-01-11 CHVG                         39.60
2000-01-12 CHVG                         41.24
2000-01-13 CHVG                         38.62
2000-01-14 CHVG                         42.28
2000-01-15 CHVG                         46.62
2000-01-16 CHVG                         50.49
2000-01-17 CHVG                         43.53
2000-01-18 CHVG                         42.08
2000-01-19 CHVG                         38.41
2000-01-20 CHVG                   